In [1]:
import json
import pandas as pd
from tqdm.notebook import tqdm
from transformers import MarianTokenizer, MarianMTModel
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
langs = ['de', 'es', 'zh', 'it', 'ro', 'vi', 'ru', 'fr', 'cs', 'jap']

In [3]:
train.iloc[151]

NameError: name 'train' is not defined

In [4]:
train = pd.read_csv("../../data/at/dbpedia/task1_dbpedia_train_translated.csv", sep='$')

In [5]:
train.head()

,id,question,category,type,question_de,question_es,question_zh,question_it,question_ro,question_vi,question_ru,question_fr,question_cs,question_jap
0,0,Was Jacqueline Kennedy Onassis a follower of M...,boolean,['boolean'],War Jacqueline Kennedy Onassis eine Anhängerin...,¿Fue Jacqueline Kennedy Onassis una seguidora ...,杰奎琳·肯尼迪·奥纳西斯是梅尔基特希腊天主教会的追随者吗?,Jacqueline Kennedy Onassis era un seguace dell...,A fost Jacqueline Kennedy Onassis un adept al ...,Phải chăng ông Kennedy là một môn đồ của Giáo ...,Была ли Жаклин Кеннеди Онассис последователем ...,Jacqueline Kennedy Onassis était - elle une di...,Byla Jacqueline Kennedy Onassisová následovník...,"アビトリダ と い う 者 が, ギリシヤ 語 を 使 う パトラマびと, クレネ 人 など..."
1,1,What is the name of the opera based on Twelfth...,resource,"['dbo:Opera', 'dbo:MusicalWork', 'dbo:Work']","Wie heißt die Oper, die auf der Zwölften Nacht...",¿Cuál es el nombre de la ópera basada en la du...,十二夜歌剧的名字是什么?,Qual è il nome dell'opera basata sulla dodices...,Care este numele operei bazate pe Noaptea a 12-a?,Tên của vở kịch dựa trên ngày 12 tháng 12 là gì?,"Как называется опера, основанная на ""Двенадцат...",Quel est le nom de l'opéra basé sur la Douzièm...,Jak se jmenuje opera založená na Dvanácté noci?,ジザ の 名 は なめらか で あ る. アカジン の 上 に は 名 が あ る か.
2,2,When did Lena Horne receive the Grammy Award f...,literal,['date'],Wann erhielt Lena Horne den Grammy Award für d...,¿Cuándo recibió Lena Horne el Premio Grammy al...,莉娜·荷恩什么时候得到 格莱美奖奖 最佳爵士Vocal专辑?,Quando Lena Horne ha ricevuto il Grammy Award ...,Când a primit Lena Horne premiul Grammy pentru...,Khi nào chị ấy nhận được giải thưởng cho bản n...,Когда Лена Хорн получила премию Грэмми за лучш...,Quand Lena Horne a-t-elle reçu le Grammy Award...,Kdy dostala Lena Horne cenu Grammy za nejlepší...,"ホルマククミテを受けるば,クミンが危険をみてきたか."
3,3,Do Prince Harry and Prince William have the sa...,boolean,['boolean'],Haben Prinz Harry und Prinz William die gleich...,¿El príncipe Harry y el príncipe Guillermo tie...,哈利王子和威廉王子有同父母吗?,Il principe Harry e il principe William hanno ...,Prinţul Harry şi prinţul William au aceiaşi pă...,Có phải Hoàng tử Harry và Hoàng tử William cũn...,У принца Гарри и принца Уильяма одни и те же р...,Le prince Harry et le prince William ont - ils...,Mají princ Harry a princ William stejné rodiče?,"君 た る 者, 君 た る 者 も, 同様 に 両親 を 持 つ だ ろ う か."
4,5,Which is the hierarchical BrainInfo ID of the ...,literal,['string'],Welches ist die hierarchische BrainInfo ID des...,¿Cuál es el ID de BrainInfo jerárquico del lób...,哪个是前额叶的大脑信息识别码?,Qual è l'ID gerarchico BrainInfo del lobo fron...,Care este ierarhicul BrainInfo ID al lobului f...,Cái nào là chứng minh của khối u?,Какой иерархический идентификатор мозга лобной...,Quel est le BrainInfo ID hiérarchique du lobe ...,Který je hierarchický BrainInfo ID čelního lal...,""" わたし は 東 に あ っ て 語 る べ き こと を 見 た こと が あ る か...."


In [ ]:
for lang in langs:
    model = MarianMTModel.from_pretrained(f'Helsinki-NLP/opus-mt-en-{lang}').to(device)
    tokenizer = MarianTokenizer.from_pretrained(f'Helsinki-NLP/opus-mt-en-{lang}')
    translation_list = list()
    
    for text in tqdm(train.question.values):
        # Tokenize the text
        batch = tokenizer([text], return_tensors="pt", padding=True).to(device)

        # Make sure that the tokenized text does not exceed the maximum
        # allowed size of 512
        batch["input_ids"] = batch["input_ids"][:, :512]
        batch["attention_mask"] = batch["attention_mask"][:, :512]
        # Perform the translation and decode the output
        translation = model.generate(**batch)
        translation_list.append(tokenizer.batch_decode(translation, skip_special_tokens=True)[0])
        
    train[f'question_{lang}'] = translation_list
    train.to_csv("../../data/at-dbpedia/dbpedia/task1_dbpedia_train_translated.csv", sep='$', index=False)

  0%|          | 0/37061 [00:00<?, ?it/s]

  0%|          | 0/37061 [00:00<?, ?it/s]

  0%|          | 0/37061 [00:00<?, ?it/s]

  0%|          | 0/37061 [00:00<?, ?it/s]

  0%|          | 0/37061 [00:00<?, ?it/s]

  0%|          | 0/37061 [00:00<?, ?it/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

